In [4]:
import copy
import itertools

import numpy as np
import plotly.offline as pty
import plotly.graph_objs as ptygo
import plotly.grid_objs as ptygr
import plotly.tools as ptytools
import scipy.stats as st

import FilterPointCloud
import ProcessPlanePointCloud

pty.init_notebook_mode(connected=True)

In [2]:
pcDataNear01 = ProcessPlanePointCloud.main('RealSense-D415-Data-01-Near/Laser/', {
	2.5: [
		'x<1.1447',
		'y<0.58',
	]
})

Processing folder RealSense-D415-Plane-0.5m-1280x720-6-Laser
Filters: []
n = 4
e = [0.001272744066909378, 0.0011975697212374943, 0.0012006031117035895, 0.0011872592474881914]
d = [3599376.6022536946, 3673032.5327609642, 3674125.838600413, 3689355.2226503114]

Processing folder RealSense-D415-Plane-0.5m-640x480-6-Laser
Filters: []
n = 5
e = [0.000979629464228184, 0.00099644149422773, 0.0009896044646350625, 0.0009927092013700958, 0.0009845015566676638]
d = [1620227.3117891802, 1625838.3362126236, 1625583.7795850907, 1623391.8987443077, 1626158.7817598465]

Processing folder RealSense-D415-Plane-1.0m-1280x720-6-Laser
Filters: []
n = 5
e = [0.00417153546403536, 0.003960580518788105, 0.00392537073043927, 0.0038850551540389323, 0.003886084783745981]
d = [871253.1868198893, 869398.9876051244, 866467.4519292535, 866021.1626674612, 870413.9585805516]

Processing folder RealSense-D415-Plane-1.0m-640x480-6-Laser
Filters: []
n = 5
e = [0.0022913510930074885, 0.00220443750579066, 0.0022464648665937

In [3]:
pcDataNear02 = ProcessPlanePointCloud.main('RealSense-D415-Data-02-Near/Laser', {
	2.0: [
		'x>-1.03',
		'x<1.26',
	],
	2.5: [
		'x>-0.742',
		'x<1.26',
		'y>-0.88',
	],
})

Processing folder RealSense-D415-Plane-0.5m-1280x720-6-Laser
Filters: []
n = 10
e = [0.0013949284095392887, 0.0014255667141571285, 0.0014325412140112567, 0.001424911219787964, 0.0014376540086578939, 0.0014199190491987416, 0.0014235292373592017, 0.0014179731787770987, 0.0014206610291585856, 0.0014177412222253422]
d = [3356355.6655147043, 3405366.25660619, 3382880.3074755417, 3396178.837123977, 3390912.999696959, 3393509.2763026934, 3405477.7994176955, 3386404.792975015, 3390380.075049065, 3395075.552176819]

Processing folder RealSense-D415-Plane-0.5m-424x240-6-Laser
Filters: []
n = 10
e = [0.001194218248898222, 0.0011666443297351232, 0.0011564655045599578, 0.0011361361898958547, 0.0011375168220738232, 0.0011248295070419789, 0.0011350264892422753, 0.0011445630060911932, 0.001132419472382647, 0.0011203402228676777]
d = [382867.72502697434, 383337.8710542158, 383505.1233342805, 383339.09719897225, 382607.2879539593, 383328.1836325655, 383189.1568733427, 383657.00266254885, 383155.58534910

In [5]:
def ptyColorMap(names):
    ptyColors = itertools.cycle(ptytools.DEFAULT_PLOTLY_COLORS)

    return dict(zip(names, ptyColors))

In [6]:
def createGraphObjectsFromPlaneData(data):
    goData = {}
    layoutData = {}
    
    colorMap = ptyColorMap(data.keys())
    
    measurandMap = {
        'planeFitError': {
            'xaxis': {
                'title': 'Distance [m]',
            },
            'yaxis': {
                'title': 'Error [m]',
            },
            'title': 'Plane Fit Error',
        },
        'pointDensity': {
            'xaxis': {
                'title': 'Distance [m]',
            },
            'yaxis': {
                'title': 'Point Density [p / m²]',
            },
            'title': 'Point Density',
        },
    }
    
    for res in data:
        showlegend = True
        
        for measurand in data[res]:
            go = ptygo.Scatter(
                x=list(data[res][measurand].keys()),
                y=[np.mean(data[res][measurand][dist]) for dist in data[res][measurand]],
                error_y=dict(
                    type='data',
                    visible=True,
                    symmetric=True,
                    array=np.abs(np.array([st.t.interval(0.95, len(data[res][measurand][dist])-1, loc=0, scale=st.sem(data[res][measurand][dist])) for dist in data[res][measurand]])[:, 0]),
                    color=colorMap[res],
                ),
                name=res,
                legendgroup=res,
                showlegend=showlegend,
                line=dict(
                    color=colorMap[res],
                ),
            )
            
            if measurand not in goData:
                goData[measurand] = []
            
            goData[measurand].append(go)
            showlegend = False
            
            if measurand in measurand:
                layoutData[measurand] = copy.deepcopy(measurandMap[measurand])
            else:
                layoutData[measurand] = {
                    'title': measurand,
                }
    
    return goData, layoutData

In [7]:
def plotPlaneData(data):
    go, lay = createGraphObjectsFromPlaneData(data)
    goKeys = [*go.keys()]
    rows = int(np.floor(np.sqrt(len(go))))
    cols = int(np.ceil(np.sqrt(len(go))))
    
    titles = tuple(lay[goKey]['title'] for goKey in goKeys)
    
    fig = ptytools.make_subplots(rows=rows, cols=cols, subplot_titles=titles, print_grid=False)
    
    for v in range(rows):
        for u in range(cols):
            i = v * rows + u
            
            if i >= len(go):
                break
            
            for t in go[goKeys[i]]:
                fig.append_trace(t, v + 1, u + 1)
            
            layout = lay[goKeys[i]]
            
            if 'xaxis' in layout:
                fig['layout']['xaxis{:d}'.format((v+1)*(rows-1)+u+1)].update(layout['xaxis'])
            
            if 'yaxis' in layout:
                fig['layout']['yaxis{:d}'.format((v+1)*(rows-1)+u+1)].update(layout['yaxis'])
    
    
    pty.iplot(fig)

In [8]:
plotPlaneData(pcDataNear01)

In [9]:
plotPlaneData(pcDataNear02)